
# Análise de Dados - Olist

Este notebook apresenta uma análise detalhada dos dados da plataforma Olist. O objetivo é identificar tendências, desafios e oportunidades para melhorar a performance de vendas, logística e satisfação dos clientes.

---

## **Sumário**
1. [Carregamento de dados](#Carregamento-de-dados)
2. [Limpeza e Tratamento dos Dados](#Limpeza-e-Tratamento-dos-Dados)
3. [Análise Exploratória de Dados (EDA)](#Análise-Exploratória-de-Dados)
    - [3.1 placeholder](#placeholder)
---


## **1. Carregamento dos Dados**

Para iniciar, carregamos os arquivos principais do dataset do Olist. Isso inclui informações sobre clientes, pedidos, vendedores, produtos e outros. A pré-visualização dos dados permite entender a estrutura e os campos disponíveis.

In [1]:

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 55)

caminhos = {
    "clientes": "dados/olist_customers_dataset.csv",
    "geolocalizacao": "dados/olist_geolocation_dataset.csv",
    "itens_pedidos": "dados/olist_order_items_dataset.csv",
    "pagamentos_pedidos": "dados/olist_order_payments_dataset.csv",
    "avaliacoes_pedidos": "dados/olist_order_reviews_dataset.csv",
    "pedidos": "dados/olist_orders_dataset.csv",
    "produtos": "dados/olist_products_dataset.csv",
    "vendedores": "dados/olist_sellers_dataset.csv",
    "traducao_categorias": "dados/product_category_name_translation.csv",
}

dataframes = {nome: pd.read_csv(caminho) for nome, caminho in caminhos.items()}

clientes_df = dataframes["clientes"]
geolocalizacao_df = dataframes["geolocalizacao"]
itens_pedidos_df = dataframes["itens_pedidos"]
pagamentos_pedidos_df = dataframes["pagamentos_pedidos"]
avaliacoes_pedidos_df = dataframes["avaliacoes_pedidos"]
pedidos_df = dataframes["pedidos"]
produtos_df = dataframes["produtos"]
vendedores_df = dataframes["vendedores"]
traducao_categorias_df = dataframes["traducao_categorias"]

### 1.1 Pré-visualização dos Dados

Exibimos uma amostra de cada DataFrame para entender sua estrutura e conteúdo.

In [2]:
clientes_df.sample(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
53451,d16c4d8784780fd06b8690d1dfd8b327,0a146272aaf254020ecc482c21f2e463,35160,ipatinga,MG
79858,21ab582841cf9dd27cf3e43dd410a472,7585f9f0cde0395dc3c840d0611420c1,11070,santos,SP


In [3]:
geolocalizacao_df.sample(2)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
584245,35051,-18.870177,-41.976243,governador valadares,MG
702158,48500,-10.502572,-39.011117,euclides da cunha,BA


In [4]:
itens_pedidos_df.sample(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
36129,51eb3e42940efb2afe2e7546e8bf2f5d,1,82183967d9fde935fc0b0189bb68ccec,6cd68b3ed6d59aaa9fece558ad360c0a,2017-02-12 06:48:03,55.9,20.84
100016,e2d97f385b79fa2f33a37c2c77965aec,1,24c66f106f642621e524291a895c9032,620c87c171fb2a6dd6e8bb4dec959fc6,2017-12-27 23:11:37,159.9,18.37


In [5]:
pagamentos_pedidos_df.sample(2)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
79180,3c4bb13fb00d3bc789b3dca0789480a5,1,credit_card,5,51.82
91156,5b26d9049a86fab0a83280e17de710ee,2,voucher,1,50.00


In [6]:
avaliacoes_pedidos_df.sample(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
56187,9680dd133ded17f631cf663dba706161,2c59b65408953b66cc6cac915015d8f9,5,NaN,NaN,2018-03-01 00:00:00,2018-03-02 14:48:11
53523,e637e38e9a2a01a9139969e298722f2b,c42fc2236969fb19c13d8ca77e1d4133,5,NaN,Produto de qualidade decora bem o ambiente,2017-08-09 00:00:00,2017-08-12 02:47:39


In [7]:
pedidos_df.sample(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
10270,5e7715473075848a1953230996ea3b3d,b08c2a9e3021776272b765210816f89a,delivered,2017-12-26 20:58:37,2017-12-26 21:09:26,2017-12-27 20:51:16,2018-01-06 12:18:52,2018-01-24 00:00:00
3359,ec67500767e46c56db2d046d13e46296,6f33f93011237ac8e47e0652b5da71de,delivered,2018-02-28 09:40:36,2018-02-28 10:40:35,2018-02-28 19:56:46,2018-03-08 18:46:42,2018-03-20 00:00:00


In [8]:
produtos_df.sample(2)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
30118,bbbf1bcea60c934fa6237fe10476c475,papelaria,48.0,533.0,1.0,1100.0,36.0,29.0,33.0
6240,999ad46c08ea9b778cefc75048156f30,informatica_acessorios,36.0,983.0,4.0,1700.0,31.0,12.0,22.0


In [9]:
vendedores_df.sample(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
2387,61f159ef6da2d441951d2c0efa719362,29160,serra,ES
2216,be8e909810184b9b19e886129149d8e5,13418,piracicaba,SP
742,0f7eea709efdf942b5f95c7516dbacd7,19865,pedrinhas paulista,SP
855,e908c0f3646e8b60375734a350d95d71,1226,sao paulo,SP
205,3db66a856d18a9cba7c9241fc5221c50,35430,ponte nova,MG


In [10]:
traducao_categorias_df.sample(2)

,product_category_name,product_category_name_english
18,eletroportateis,small_appliances
39,livros_tecnicos,books_technical


In [11]:
produtos_df.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


### 1.2 Identificação de valores ausentes

Identificamos valores ausentes em cada coluna dos DataFrames e organizamos os resultados em uma tabela consolidada para melhor visualização.

In [12]:
resumo = []

for nome, df in dataframes.items():
    valores_ausentes = df.isnull().sum() 
    df_resumo = pd.DataFrame({
        "DataFrame": [nome] * len(valores_ausentes),
        "Coluna": valores_ausentes.index,
        "Valores ausentes": valores_ausentes.values
    })
    resumo.append(df_resumo)
    
df_resumo_final = pd.concat(resumo, ignore_index=True)
df_resumo_final


,DataFrame,Coluna,Valores ausentes
0,clientes,customer_id,0
1,clientes,customer_unique_id,0
2,clientes,customer_zip_code_prefix,0
3,clientes,customer_city,0
4,clientes,customer_state,0
5,geolocalizacao,geolocation_zip_code_prefix,0
6,geolocalizacao,geolocation_lat,0
7,geolocalizacao,geolocation_lng,0
8,geolocalizacao,geolocation_city,0
9,geolocalizacao,geolocation_state,0


## **2. Limpeza e Tratamento dos Dados**
Antes de realizar análises mais aprofundadas, é essencial tratar dados ausentes e inconsistências. Nesta etapa:
- Comentários e títulos ausentes nas avaliações foram preenchidos com valores padrão.
- Valores numéricos ausentes (como dimensões de produtos) foram preenchidos com a média.

In [13]:

avaliacoes_pedidos_df['review_comment_message'] = avaliacoes_pedidos_df['review_comment_message'].fillna('Sem Comentário')
avaliacoes_pedidos_df['review_comment_title'] = avaliacoes_pedidos_df['review_comment_title'].fillna('Sem Título')

colunas_numericas = [
    'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
    'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'
]
produtos_df[colunas_numericas] = produtos_df[colunas_numericas].apply(lambda col: col.fillna(col.mean()))

produtos_df.isnull().sum()


product_id                      0
product_category_name         610
product_name_lenght             0
product_description_lenght      0
product_photos_qty              0
product_weight_g                0
product_length_cm               0
product_height_cm               0
product_width_cm                0
dtype: int64

## **3. Análise Exploratória de Dados (EDA)**

Exploramos os dados disponíveis para identificar tendências, padrões e insights relevantes. A análise abrange diferentes aspectos da plataforma Olist, como categorias de produtos, métodos de pagamento e outros. Essas informações ajudam a compreender o comportamento dos clientes, desempenho de vendas e possíveis áreas de melhoria.


### 3.1 Top 10 Categorias Mais Vendidas

In [14]:
produtos_df = produtos_df.merge(traducao_categorias_df, on='product_category_name', how='left')

top_categorias = (
    itens_pedidos_df.merge(produtos_df, on='product_id')
    .groupby('product_category_name_english')['order_id']
    .count()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'order_id': 'Quantidade de Pedidos', 'product_category_name_english': 'Categoria'})
    .head(10)
)
top_categorias

,Categoria,Quantidade de Pedidos
0,bed_bath_table,11115
1,health_beauty,9670
2,sports_leisure,8641
3,furniture_decor,8334
4,computers_accessories,7827
5,housewares,6964
6,watches_gifts,5991
7,telephony,4545
8,garden_tools,4347
9,auto,4235


In [15]:
fig1 = px.bar(
    top_categorias,
    x='Quantidade de Pedidos',
    y='Categoria',
    orientation='h',
    title='Top 10 Categorias Mais Vendidas',
    labels={'Quantidade de Pedidos': 'Pedidos', 'Categoria': 'Categoria'},
    text='Quantidade de Pedidos'
)
fig1.update_layout(title_x=0.5)
fig1.show()

### 3.2 Métodos de Pagamento Mais Utilizados

In [16]:
metodos_pagamento = (
    pagamentos_pedidos_df.groupby('payment_type')['payment_value']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'payment_type': 'Método de Pagamento', 'payment_value': 'Total Pago (R$)'})
)
metodos_pagamento

,Método de Pagamento,Total Pago (R$)
0,credit_card,12542084.19
1,boleto,2869361.27
2,voucher,379436.87
3,debit_card,217989.79
4,not_defined,0.00


In [17]:
fig2 = px.pie(
    metodos_pagamento,
    values='Total Pago (R$)',
    names='Método de Pagamento',
    title='Distribuição dos Pagamentos por Método',
    hole=0.4
)
fig2.update_layout(title_x=0.5)
fig2.show()

### 3.3 Tempo médio de entrega em cada estado

In [18]:
pedidos_df['order_delivered_customer_date'] = pd.to_datetime(pedidos_df['order_delivered_customer_date'], errors='coerce')
pedidos_df['order_estimated_delivery_date'] = pd.to_datetime(pedidos_df['order_estimated_delivery_date'], errors='coerce')
pedidos_df['order_purchase_timestamp'] = pd.to_datetime(pedidos_df['order_purchase_timestamp'], errors='coerce')

pedidos_df['Tempo de Entrega (dias)'] = (pedidos_df['order_delivered_customer_date'] - pedidos_df['order_purchase_timestamp']).dt.days

pedidos_df['Atraso na Entrega'] = pedidos_df['order_delivered_customer_date'] > pedidos_df['order_estimated_delivery_date']

clientes_com_entrega = clientes_df.merge(pedidos_df, on='customer_id')
media_entrega_por_estado = (
    clientes_com_entrega.groupby('customer_state')['Tempo de Entrega (dias)']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'customer_state': 'Estado', 'Tempo de Entrega (dias)': 'Média de Entrega (dias)'})
)

media_entrega_por_estado

,Estado,Média de Entrega (dias)
0,RR,28.975610
1,AP,26.731343
2,AM,25.986207
3,AL,24.040302
4,PA,23.316068
5,MA,21.117155
6,SE,21.029851
7,CE,20.817826
8,AC,20.637500
9,PB,19.953578


In [19]:
fig4 = px.bar(
    media_entrega_por_estado,
    x='Média de Entrega (dias)',
    y='Estado',
    orientation='h',
    title='Média de Tempo de Entrega por Estado',
    labels={'Estado': 'Estado', 'Média de Entrega (dias)': 'Tempo Médio (dias)'},
    text='Média de Entrega (dias)',
    color='Média de Entrega (dias)',
    color_continuous_scale=px.colors.sequential.Plasma
)

fig4.update_traces(
    texttemplate='%{text:.2f}', 
    textposition='outside', 
    marker_line_color='black', 
    marker_line_width=1.5
)

fig4.update_layout(
    title={
        'text': 'Média de Tempo de Entrega por Estado',
        'x': 0.5,
        'xanchor': 'center'
    },
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="#000"
    ),
    xaxis=dict(
        title='Tempo Médio (dias)',
        showgrid=True,
        gridcolor='lightgrey',
        zeroline=True,
        zerolinecolor='black'
    ),
    yaxis=dict(
        title='Estado',
        showgrid=False
    ),
    coloraxis_colorbar=dict(
        title='Tempo Médio (dias)',
        thickness=15
    ),
    height=700,
    margin=dict(l=150, r=50, t=70, b=50)  # Ajustando as margens para visibilidade
)

fig4.show()